In [1]:
from datetime import datetime, timedelta
import json
import uuid
import random 
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import cultpass

# Udahub Accounts

## Cultpass Database

**Init DB**

In [2]:
cultpass_db = "data/external/cultpass.db"

In [3]:
reset_db(cultpass_db)

✅ Removed existing data/external/cultpass.db
2025-11-10 10:00:45,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-10 10:00:45,117 INFO sqlalchemy.engine.Engine COMMIT
✅ Recreated data/external/cultpass.db with fresh schema


In [4]:
engine = create_engine(f"sqlite:///{cultpass_db}", echo=False)
cultpass.Base.metadata.create_all(engine)

**Experiences**

In [5]:
experience_data = []

with open('data/external/cultpass_experiences.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        experience_data.append(json.loads(line))

In [6]:
experience_data

[{'title': 'Carnival History Tour in Olinda',
  'description': "Discover the origins and vibrant traditions of Pernambuco's Carnival.",
  'location': 'Pernambuco, Brazil'},
 {'title': 'Sunset Paddleboarding',
  'description': 'Glide across calm waters at golden hour with all gear included.',
  'location': 'Santa Catarina, Brazil'},
 {'title': 'Pelourinho Colonial Walk',
  'description': 'Wander through colorful streets and learn about Afro-Brazilian history.',
  'location': 'Bahia, Brazil'},
 {'title': 'Samba Night at Lapa',
  'description': 'Dance the night away at a traditional samba club in the Lapa arches.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Christ the Redeemer Experience',
  'description': 'Take a guided trip to one of the New Seven Wonders of the World with historical context.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Modern Art at MASP',
  'description': 'Enjoy a guided visit to the São Paulo Museum of Art with insights into its top collections.',

In [7]:
with get_session(engine) as session:
    experiences = []

    for idx, experience in enumerate(experience_data):
        exp = cultpass.Experience(
            experience_id=str(uuid.uuid4())[:6],
            title=experience["title"],
            description=experience["description"],
            location=experience["location"],
            when=datetime.now() + timedelta(days=idx+1),
            slots_available=random.randint(1,30),
            is_premium=(idx % 2 == 0)
        )
        experiences.append(exp)

    session.add_all(experiences)

**User**

In [8]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [9]:
cultpass_users

[{'id': 'a4ab87',
  'name': 'Alice Kingsley',
  'email': 'alice.kingsley@wonderland.com',
  'is_blocked': True},
 {'id': 'f556c0',
  'name': 'Bob Stone',
  'email': 'bob.stone@granite.com',
  'is_blocked': False},
 {'id': '88382b',
  'name': 'Cathy Bloom',
  'email': 'cathy.bloom@florals.org',
  'is_blocked': False},
 {'id': '888fb2',
  'name': 'David Noir',
  'email': 'david.noir@shadowmail.com',
  'is_blocked': True},
 {'id': 'f1f10d',
  'name': 'Eva Green',
  'email': 'eva.green@ecosoul.net',
  'is_blocked': False},
 {'id': 'e6376d',
  'name': 'Frank Ocean',
  'email': 'frank.ocean@seawaves.io',
  'is_blocked': False}]

In [10]:
with get_session(engine) as session:
    db_users = []
    for user_info in cultpass_users:
        user = cultpass.User(
            user_id=user_info["id"],
            full_name=user_info["name"],
            email=user_info["email"],
            is_blocked=user_info["is_blocked"],
            created_at=datetime.now()
        )
        db_users.append(user)
    session.add_all(db_users) 

**Subscription**

In [11]:
with get_session(engine) as session:
    subscriptions = []
    for user_info in cultpass_users:
        subscription = cultpass.Subscription(
            subscription_id=str(uuid.uuid4())[:6],
            user_id=user_info["id"],
            status=random.choice(["active", "cancelled"]),
            tier=random.choice(["basic", "premium"]),
            monthly_quota=random.randint(2,10),
            started_at=datetime.now()
        )
        subscriptions.append(subscription)

    session.add_all(subscriptions)

**Reservation**

In [12]:
# Applicable to `cultpass_users[0]` at the moment

with get_session(engine) as session:
    experience_ids = [
        exp.experience_id 
        for exp 
        in session.query(cultpass.Experience).all()
    ]

    reservation1 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    reservation2 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    session.add_all([reservation1, reservation2])

In [13]:
# Add more reservation data for multiple users
# Please notice that the reservations were set to first user only 
# If you want to simulate more users later, please create more reservations per user

with get_session(engine) as session:
    experience_ids = [
        exp.experience_id 
        for exp 
        in session.query(cultpass.Experience).all()
    ]
    
    # Get all users
    all_users = session.query(cultpass.User).all()
    
    # Create reservations for multiple users (not just the first one)
    additional_reservations = []
    
    # Add 1-2 reservations for each user (except first user who already has 2)
    for user in all_users[1:]:  # Skip first user
        num_reservations = random.randint(1, 2)
        for _ in range(num_reservations):
            reservation = cultpass.Reservation(
                reservation_id=str(uuid.uuid4())[:6],
                user_id=user.user_id,
                experience_id=random.choice(experience_ids),
                status=random.choice(["reserved", "cancelled", "completed"]),
            )
            additional_reservations.append(reservation)
    
    # Add a few more reservations for the first user as well
    for _ in range(2):
        reservation = cultpass.Reservation(
            reservation_id=str(uuid.uuid4())[:6],
            user_id=all_users[0].user_id,
            experience_id=random.choice(experience_ids),
            status=random.choice(["reserved", "cancelled", "completed"]),
        )
        additional_reservations.append(reservation)
    
    session.add_all(additional_reservations)

# Tests

In [14]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user)

<User(user_id='a4ab87', email='alice.kingsley@wonderland.com', is_blocked=True)>
<User(user_id='f556c0', email='bob.stone@granite.com', is_blocked=False)>
<User(user_id='88382b', email='cathy.bloom@florals.org', is_blocked=False)>
<User(user_id='888fb2', email='david.noir@shadowmail.com', is_blocked=True)>
<User(user_id='f1f10d', email='eva.green@ecosoul.net', is_blocked=False)>
<User(user_id='e6376d', email='frank.ocean@seawaves.io', is_blocked=False)>


In [15]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user.subscription)

<Subscription(subscription_id='efa03e', user_id='a4ab87', status='active', tier='premium')>
<Subscription(subscription_id='e7f865', user_id='f556c0', status='cancelled', tier='premium')>
<Subscription(subscription_id='6c5b4e', user_id='88382b', status='cancelled', tier='premium')>
<Subscription(subscription_id='9f75b2', user_id='888fb2', status='active', tier='premium')>
<Subscription(subscription_id='e34481', user_id='f1f10d', status='active', tier='basic')>
<Subscription(subscription_id='c9575d', user_id='e6376d', status='cancelled', tier='premium')>


In [16]:
with get_session(engine) as session:
    experiences = session.query(cultpass.Experience).all()
    for experience in experiences:
        print(experience)

<Experience(experience_id='afbe2d', title='Carnival History Tour in Olinda', when='2025-11-11 10:00:45.191705')>
<Experience(experience_id='ec14f1', title='Sunset Paddleboarding', when='2025-11-12 10:00:45.263595')>
<Experience(experience_id='c52908', title='Pelourinho Colonial Walk', when='2025-11-13 10:00:45.263645')>
<Experience(experience_id='fdf895', title='Samba Night at Lapa', when='2025-11-14 10:00:45.263673')>
<Experience(experience_id='b9d993', title='Christ the Redeemer Experience', when='2025-11-15 10:00:45.263715')>
<Experience(experience_id='b72e5a', title='Modern Art at MASP', when='2025-11-16 10:00:45.263750')>
<Experience(experience_id='278fee', title='Ibirapuera Park Bike Ride', when='2025-11-17 10:00:45.263773')>
